# REFINITIV COMPANY TEARSHEET

In [546]:
RIC = input()
# print(RIC)

IBM


In [631]:
# Import module and set up with App Key
import eikon as ek
ek.set_app_key('8a0a51d096f34eec86dadab1763ad94dae81c80e')
import pandas as pd 
import numpy as np

In [638]:
# Request Header Information for RIC 
df, err = ek.get_data(RIC, "TR.CommonName(),TR.HeadquartersCountry,TR.TRBCIndustryGroup,TR.GICSSector,TR.CompanyMarketCap.Currency")
#Request Exchange for Primary Quote RIC
df_2, err = ek.get_data(RIC,"TR.PrimaryQuote")
df_3, err = ek.get_data(df_2.at[0,'Primary Quote RIC'],"TR.ExchangeName")  
df['Exchange Name'] = [df_3.at[0,'Exchange Name']]
df

,Instrument,Company Common Name,Country of Headquarters,TRBC Industry Group Name,GICS Sector Name,Currency,Exchange Name
0,IBM,International Business Machines Corp,United States of America,Software & IT Services,Information Technology,USD,"NEW YORK STOCK EXCHANGE, INC."


## General Information

In [673]:
#As single Dataframe - example only
#dfGenSingle, err = ek.get_data(RIC, 
#        [{'TR.PriceClose;TR.Price52WeekHigh;TR.PricePctChg52WkHigh;TR.Price52WeekLow;TR.PricePctChg52WkLow;TR.PriceTargetMedian': {'Curn':df.at[0,'Currency']}},
#            {'TR.BetaFiveYear;TR.DividendYield;TR.SharesOutstanding;TR.FreeFloat;TR.FreeFloatPct();TR.CompanyMarketCap;TR.EV': {'Scale':6,'Curn':df.at[0,'Currency']}}])

#As two Dataframes - more convenient
dfGen, err = ek.get_data(RIC, 'TR.PriceClose;TR.Price52WeekHigh;TR.PricePctChg52WkHigh;TR.Price52WeekLow;TR.PricePctChg52WkLow;TR.PriceTargetMedian',{'Curn':df.at[0,'Currency']})
dfGen2, err = ek.get_data( RIC, 'TR.BetaFiveYear;TR.DividendYield;TR.SharesOutstanding;TR.FreeFloat;TR.FreeFloatPct();TR.CompanyMarketCap;TR.EV',{'Scale':6,'Curn':df.at[0,'Currency']})

#calc implied profit
dfGen['Implied Profit'] = (dfGen.at[0,'Price Target - Median']/dfGen.at[0,'Price Close'] - 1)*100

# results as simple lists 
listGenNames = dfGen.columns.tolist()
listGen = dfGen.iloc[0].tolist()
listGen2Names = dfGen2.columns.tolist()
listGen2 = dfGen2.iloc[0].tolist()

#remove RIC
listGenNames.pop(0)
listGen.pop(0)
listGen2Names.pop(0)
listGen2.pop(0)

dataDictGen= {'1': listGenNames, '2': listGen , '3': listGen2Names, '4': listGen2} 
dfGenX = pd.DataFrame(dataDictGen)  

dfGenX

,1,2,3,4
0,Price Close,110.930000,Beta 5 Year,1.295649
1,Price 52 Week High,158.750000,Dividend yield,5.738068
2,52-week High Price PCT Change,-30.122835,Outstanding Shares,888.408023
3,Price 52 Week Low,90.560000,Free Float,887.346656
4,52-week Low Price PCT Change,22.493375,Free Float (Percent),99.880500
5,Price Target - Median,139.000000,Company Market Cap,98551.101991
6,Implied Profit,25.304246,Enterprise Value (Daily Time Series),152726.101991


## Consensus Recommendation

In [660]:
df6, err = ek.get_data(RIC, 'TR.RecLabel();TR.NumOfRecommendations();TR.NumEstRevisingUp;TR.NumEstRevisingDown',{'WP':'30d','EstimateMeasure':'EPS'})
df6

,Instrument,Recommendation - Mean Label,Recommendation - Number Of Total,Number of Analysts Revising Up,Number of Analysts Revising Down
0,IBM,HOLD,20,0,5


## Estimates

In [588]:
#Retrieve Estimates into Data Frames per Period

#FY0
dfFY0, err = ek.get_data(RIC, 'TR.RevenueActValue;TR.EBITDAActValue;TR.EPSActValue',
                       {'Period':'FY0', 'Scale':'6', 'Curn':df.at[0,'Currency']})
#FY1
dfFY1, err = ek.get_data(RIC, 
'TR.RevenueMeanEstimate;TR.RevenueHigh;TR.RevenueLow;TR.EBITDAMean;TR.EBITDAHigh;TR.EBITDALow;TR.EPSMeanEstimate;TR.EPSHigh;TR.EPSLow',
                       {'Period':'FY1', 'scale':'6', 'curn':df.at[0,'Currency']})
dfFY1_2, err = ek.get_data(RIC, 
                           'TR.RevenueMeanEstimate; TR.EBITDAMean; TR.EPSMeanEstimate',
    {'Period':'FY1','RollPeriods':'false','Scale': 6,'Curn':df.at[0,'Currency'],'SDate':'-1AM'})
#FY2
dfFY2, err = ek.get_data(RIC, 
'TR.RevenueMeanEstimate;TR.RevenueHigh;TR.RevenueLow;TR.EBITDAMean;TR.EBITDAHigh;TR.EBITDALow;TR.EPSMeanEstimate;TR.EPSHigh;TR.EPSLow',
                       {'Period':'FY2', 'scale':'6', 'curn':df.at[0,'Currency']})
dfFY2_2, err = ek.get_data(RIC, 'TR.RevenueMeanEstimate;TR.EBITDAMean;TR.EPSMeanEstimate',
    {'Period':'FY2','RollPeriods':'false','Scale': 6,'Curn':df.at[0,'Currency'],'SDate':'-1AM'})

### DEBUG display(dfFY0,dfFY1,dfFY1_2, dfFY2, dfFY2_2)

In [589]:
#Create a dictionary out of results, and structure a single multi-row Data Frame

# initialise results per column 
data = {'Sales (MIL) Mean EST':
        [dfFY0.at[0,'Revenue - Actual'], dfFY1.at[0,'Revenue - Mean Estimate'],'',dfFY2.at[0,'Revenue - Mean Estimate'],''],
       'High Est':
       ['',dfFY1.at[0,'Revenue - High'],(dfFY1.at[0,'Revenue - High']/dfFY1.at[0,'Revenue - Mean Estimate']-1)*100,
        dfFY2.at[0,'Revenue - High'],(dfFY2.at[0,'Revenue - High']/dfFY2.at[0,'Revenue - Mean Estimate']-1)*100],
       'Low Est':
       ['',dfFY1.at[0,'Revenue - Low'],(dfFY1.at[0,'Revenue - Low']/dfFY1.at[0,'Revenue - Mean Estimate']-1)*100,
        dfFY2.at[0,'Revenue - Low'],(dfFY2.at[0,'Revenue - Low']/dfFY2.at[0,'Revenue - Mean Estimate']-1)*100],
       'Est. 30 30 Days Ago':
       ['',dfFY1_2.at[0,'Revenue - Mean Estimate'],(dfFY1_2.at[0,'Revenue - Mean Estimate']/dfFY1.at[0,'Revenue - Mean Estimate']-1)*100,
        dfFY2_2.at[0,'Revenue - Mean Estimate'],(dfFY2_2.at[0,'Revenue - Mean Estimate']/dfFY2.at[0,'Revenue - Mean Estimate']-1)*100],
       'EBITDA (MIL) Mean Est.':
        [dfFY0.at[0,'EBITDA - Actual'],dfFY1.at[0,'EBITDA - Mean'],'',dfFY2.at[0,'EBITDA - Mean'],''],
        'High Est.':
        ['',dfFY1.at[0,'EBITDA - High'],(dfFY1.at[0,'EBITDA - High']/dfFY1.at[0,'EBITDA - Mean']-1)*100,
         dfFY2.at[0,'EBITDA - High'],(dfFY2.at[0,'EBITDA - High']/dfFY2.at[0,'EBITDA - Mean']-1)*100],
        'Low Est.':
        ['',dfFY1.at[0,'EBITDA - Low'],(dfFY1.at[0,'EBITDA - Low']/dfFY1.at[0,'EBITDA - Mean']-1)*100,
         dfFY2.at[0,'EBITDA - Low'],(dfFY2.at[0,'EBITDA - Low']/dfFY2.at[0,'EBITDA - Mean']-1)*100],
        'Est. 30 30 Days Ago.':
       ['',dfFY1_2.at[0,'EBITDA - Mean'],(dfFY1_2.at[0,'EBITDA - Mean']/dfFY1.at[0,'EBITDA - Mean']-1)*100,
        dfFY2_2.at[0,'EBITDA - Mean'],(dfFY2_2.at[0,'EBITDA - Mean']/dfFY2.at[0,'EBITDA - Mean']-1)*100],
        'EPS Mean Est.':
        [dfFY0.at[0,'Earnings Per Share - Actual'], dfFY1.at[0,'Earnings Per Share - Mean Estimate'],'',
         dfFY2.at[0,'Earnings Per Share - Mean Estimate'],''],
        'High Est..':
         ['',dfFY1.at[0,'Earnings Per Share - High'],(dfFY1.at[0,'Earnings Per Share - High']/dfFY1.at[0,'Earnings Per Share - Mean Estimate']-1)*100,
        dfFY2.at[0,'Earnings Per Share - High'],(dfFY2.at[0,'Earnings Per Share - High']/dfFY2.at[0,'Earnings Per Share - Mean Estimate']-1)*100],
        'Low Est..':
         ['',dfFY1.at[0,'Earnings Per Share - Low'],(dfFY1.at[0,'Earnings Per Share - Low']/dfFY1.at[0,'Earnings Per Share - Mean Estimate']-1)*100,
        dfFY2.at[0,'Earnings Per Share - Low'],(dfFY2.at[0,'Earnings Per Share - Low']/dfFY2.at[0,'Earnings Per Share - Mean Estimate']-1)*100],
        'Est. 30 30 Days Ago..':
        ['',dfFY1_2.at[0,'Earnings Per Share - Mean Estimate'],(dfFY1_2.at[0,'Earnings Per Share - Mean Estimate']/dfFY1.at[0,'Earnings Per Share - Mean Estimate']-1)*100,
        dfFY2_2.at[0,'Earnings Per Share - Mean Estimate'],(dfFY2_2.at[0,'Earnings Per Share - Mean Estimate']/dfFY2.at[0,'Earnings Per Share - Mean Estimate']-1)*100]
       }  
  
# Create Pandas DataFrame. 
dfMultiRow = pd.DataFrame(data, index =['FY0 (Actual)','FY1 (Current Year)', '% above/below mean','FY2 (Next Year)','% above/below mean' ]) 
                                             
dfMultiRow                     

,Sales (MIL) Mean EST,High Est,Low Est,Est. 30 30 Days Ago,EBITDA (MIL) Mean Est.,High Est.,Low Est.,Est. 30 30 Days Ago.,EPS Mean Est.,High Est..,Low Est..,Est. 30 30 Days Ago..
FY0 (Actual),77147,,,,19029,,,,12.81,,,
FY1 (Current Year),77774.7,80600,70901,78837.7,18803.4,21202,15346.3,19005.3,13.0821,13.62,10.9,13.3738
% above/below mean,,3.63269,-8.83791,1.36684,,12.7561,-18.3859,1.0736,,4.11173,-16.68,2.22954
FY2 (Next Year),79439.3,83400,71406,80159.2,19806.4,21997,15979.5,20015.7,14.0082,14.7,12.26,14.1772
% above/below mean,,4.98579,-10.1125,0.906241,,11.06,-19.3217,1.05661,,4.93891,-12.4795,1.20673


# Valuation Ratios

In [590]:
# Retrieve Valuation Ratios into DataFrames per period

dfLTM, err  = ek.get_data(RIC,'TR.PE;TR.PriceToSalesPerShare;TR.EVToSales;TR.EVToEBITDA',{'Sdate':'0D'})
dfFY1, err = ek.get_data(RIC, 'TR.PToEPSMeanEst;TR.PToREVMeanEst',{'Period':'FY1'})
dfFY1_2, err= ek.get_data(RIC,'TR.EV;TR.RevenueMeanEstimate,TR.EBITDAMean',{'Period':'FY1','Curn':df.at[0,'Currency']})
dfFY2, err = ek.get_data(RIC, 'TR.PToEPSMeanEst;TR.PToREVMeanEst',{'Period':'FY2'})
dfFY2_2, err= ek.get_data(RIC,'TR.EV;TR.RevenueMeanEstimate,TR.EBITDAMean',{'Period':'FY2','Curn':df.at[0,'Currency']})
###DEBUG display(dfLTM,dfFY1,dfFY1_2,dfFY2, dfFY2_2)

In [591]:
#Create a dictionary out of results, and structure a single multi-row Data Frame

# Initialise results per column 
dataDict = {'P/E':
        [dfLTM.at[0,'P/E (Daily Time Series Ratio)'], dfFY1.at[0,'Price / EPS (Mean Estimate)'], dfFY2.at[0,'Price / EPS (Mean Estimate)']],
            'P/SALES':
        [dfLTM.at[0,'Price To Sales Per Share (Daily Time Series Ratio)'], dfFY1.at[0,'Price / Revenue (Mean Estimate)'], dfFY2.at[0,'Price / Revenue (Mean Estimate)']],
            'EV/REVS':
                [dfLTM.at[0,'Enterprise Value To Sales (Daily Time Series Ratio)'], 
                 dfFY1_2.at[0,'Enterprise Value (Daily Time Series)']/dfFY1_2.at[0,'Revenue - Mean Estimate'], 
                 dfFY2_2.at[0,'Enterprise Value (Daily Time Series)']/dfFY2_2.at[0,'Revenue - Mean Estimate']],
            
            'EV/EBITDA':
                [dfLTM.at[0,'Enterprise Value To EBITDA (Daily Time Series Ratio)'], 
                 dfFY1_2.at[0,'Enterprise Value (Daily Time Series)']/dfFY1_2.at[0,'EBITDA - Mean'], 
                 dfFY2_2.at[0,'Enterprise Value (Daily Time Series)']/dfFY2_2.at[0,'EBITDA - Mean']]
}

#Create Pandas DataFrame. 
dfMultiRow = pd.DataFrame(dataDict, index =['LTM (Last Twelve Months)','FY1 (Current Year)', 'FY2 (Next Year)']) 
dfMultiRow        

,P/E,P/SALES,EV/REVS,EV/EBITDA
LTM (Last Twelve Months),10.068015,1.244034,1.946255,7.810535
FY1 (Current Year),8.632406,1.289982,1.986545,8.216743
FY2 (Next Year),8.061735,1.262950,1.944917,7.800654


## FY0 Per Share Metrics 

In [592]:
dfFYOShare, err = ek.get_data(RIC ,['TR.TotCashAndShortTermInvestmentsperShare',
'TR.WorkingCapitalperShare','BookValuePerShare','TR.RevenuePerShare','TR.CashFlowtoShrdil',
'TR.FreeOperatingCashFlowperShareAvgDilutedSharesOut','TR.DPSCommonStock'], {'Period':'FY0','curn':df.at[0,'Currency']})
dfFYOShare_2, err =  ek.get_data(RIC ,['TR.TotalDebt','TR.TtlCmnSharesOut'],{'Period':'FY0','curn':df.at[0,'Currency']})
# additionally, calculate total debt
dfFYOShare['Total Debt'] = [dfFYOShare_2.at[0,'Total Debt']/dfFYOShare_2.at[0,'Total Common Shares Outstanding']]
dfFYOShare

,Instrument,Total Cash & ST Invest/Share,Wk Cap/Shr,BOOKVALUEPERSHARE,Revenue Per Share,Cash Flow/Shr (dil.),FOCF/Shr (dil.),DPS - Common Stock Primary Issue,Total Debt
0,IBM,9.9965,0.810497,NaN,86.408875,17.517659,6.896178,6.43,70.903234


## FY0 Employee Data

In [593]:
dfEmp, err = ek.get_data(RIC,'TR.Employees(Period=FY0),TR.RevenueperEmployeescalYrEnd(Period=FY0)')
dfEmp_2, err = ek.get_data(RIC,'TR.Employees(Period=FY-1)')
dfEmp_3, err = ek.get_data(RIC,'TR.RevenueperEmployeescalYrEnd(Period=FY0,sdate=0D,Edate=-4FY,Frq=FY)')
dfEmp['Empt Grth'] = (dfEmp_2.at[0,'Full-Time Employees']-dfEmp.at[0,'Full-Time Employees'])/dfEmp_2.at[0,'Full-Time Employees']
dfEmp['5 yr Avg'] = dfEmp_3["Revenue/Employee, FY"].mean()
dfEmp

,Instrument,Full-Time Employees,"Revenue/Employee, FY",Empt Grth,5 yr Avg
0,IBM,352600,219416.951081,-0.005705,215998.501851


## Abbreviated Financials

### Income Statement (MIL)

In [615]:
dfFin, err = ek.get_data(RIC, 'TR.TotalRevenue;TR.GrossProfit;TR.EBITDA;TR.EBIT;TR.NetIncomeBeforeTaxes;TR.NetIncome;TR.TtlCmnSharesOut',{'Period':'FY0','Scale':'6','curn':df.at[0,'Currency']})
dfFin2, err = ek.get_data(RIC, 'TR.TotalRevenue;TR.GrossProfit;TR.EBITDA;TR.EBIT;TR.NetIncomeBeforeTaxes;TR.NetIncome;TR.TtlCmnSharesOut',{'Period':'FY-1','Scale':'6','curn':df.at[0,'Currency']})

# results as simple lists 
listFin = dfFin.iloc[0].tolist() #select row 0, all columns
listFin2 = dfFin2.iloc[0].tolist()

# results as single dictionary
dataDict2 = {'FY0': listFin, 'FY-1': listFin2 }

# Pandas DataFrame from dictionary 
dfIncome = pd.DataFrame(dataDict2, index = dfFin.columns) 
dfIncome        

,FY0,FY-1
Instrument,IBM,IBM
Total Revenue,77147,79591
Gross Profit,36488,36936
EBITDA,18986,17725
EBIT,12927,13245
Net Income Before Taxes,10166,11342
Net Income Incl Extra Before Distributions,9431,8728
Total Common Shares Outstanding,887.11,892.479


### Balance Sheet (MIL)

In [604]:
dfBal, err = ek.get_data(RIC, 'TR.CashAndSTInvestments;TR.PropertyPlantEquipmentTotalNet;TR.TotalAssetsReported;TR.CurrentLiabilities;TR.TotalLongTermDebt;TR.TotalLiabilities;TR.TotalEquity',{'Period':'FY0','Scale':6,'Curn':df.at[0,'Currency']})
dfBal2, err = ek.get_data(RIC, 'TR.CashAndSTInvestments;TR.PropertyPlantEquipmentTotalNet;TR.TotalAssetsReported;TR.CurrentLiabilities;TR.TotalLongTermDebt;TR.TotalLiabilities;TR.TotalEquity',{'Period':'FY-1','Scale':6,'Curn':df.at[0,'Currency']})

# results as single dictionary
dataDict3 = {'FY0': list(dfBal.iloc[0, :]), 'FY-1': list(dfBal2.iloc[0, :]) }

# Pandas DataFrame from dictionary 
dfBalance= pd.DataFrame(dataDict3, index = dfBal.columns) 
dfBalance 

,FY0,FY-1
Instrument,IBM,IBM
Cash and Short Term Investments,8868,11999
"Property/Plant/Equipment, Total - Net",15006,10792
"Total Assets, Reported",152186,123382
Current Liabilities,37701,38227
Total Long Term Debt,54102,35605
Total Liabilities,131345,106587
Total Equity,20841,16795


### Cash Flow Statement (MIL)

In [611]:
dfCash, err = ek.get_data(RIC, 'TR.NetIncomeStartingLine',{'Period':'FY0','Scale':'6','curn':df.at[0,'Currency']})
dfCash_2, err = ek.get_data(RIC, 'TR.NetIncomeStartingLine,TR.DepreciationDepletion,TR.Amortization,TR.NonCashItems,TR.DeferredTaxes',{'Period':'FY0','Scale':'6','Curn':df.at[0,'Currency']})
dfCash_3, err = ek.get_data(RIC, 'TR.CashFromOperatingAct;TR.CapitalExpenditures;TR.CashFromInvestingAct;TR.CashDividendsPaid*-1;TR.CashFromFinancingAct',{'Period':'FY0','Scale':6,'Curn':df.at[0,'Currency']})
#insert as columns, entincome and  sum of cash from operations
dfCash_3['Net Income'] = [dfCash.at[0,'Net Income/Starting Line, Cumulative']]
dfCash_3['Cash From Operations'] = [sum(list(dfCash_2.iloc[0, 1:]))]

dfCash2, err = ek.get_data(RIC, 'TR.NetIncomeStartingLine',{'Period':'FY-1','Scale':'6','curn':df.at[0,'Currency']})
dfCash2_2, err = ek.get_data(RIC, 'TR.NetIncomeStartingLine,TR.DepreciationDepletion,TR.Amortization,TR.NonCashItems,TR.DeferredTaxes',{'Period':'FY-1','Scale':'6','Curn':df.at[0,'Currency']})
dfCash2_3, err = ek.get_data(RIC, 'TR.CashFromOperatingAct;TR.CapitalExpenditures;TR.CashFromInvestingAct;TR.CashDividendsPaid*-1;TR.CashFromFinancingAct',{'Period':'FY-1','Scale':6,'Curn':df.at[0,'Currency']})
#insert as columns, entincome and  sum of cash from operations
dfCash2_3['Net Income'] = [dfCash2.at[0,'Net Income/Starting Line, Cumulative']]
dfCash2_3['Cash From Operations'] = [sum(list(dfCash2_2.iloc[0, 1:]))]

#DEBUG display(dfCash_3,dfCash2_3)

# results as simple lists
listCash = dfCash_3.iloc[0, :].tolist()
listCash2 = dfCash2_3.iloc[0, :].tolist()


# results as single dictionary
dataDict3 = {'FY0': listCash, 'FY-1': listCash2 }
#dataDict3

# Pandas DataFrame from dictionary 
dfCashFlow = pd.DataFrame(dataDict3, index = dfCash_3.columns) 
dfCashFlow  

,FY0,FY-1
Instrument,IBM,IBM
"Cash from Operating Activities, Cumulative",14771,15247
"Capital Expenditures, Cumulative",-2907,-3964
"Cash from Investing Activities, Cumulative",-26936,-4913
TR.CASHDIVIDENDSPAID*-1,-5707,-5666
"Cash from Financing Activities, Cumulative",9042,-10469
Net Income,9431,8728
Cash From Operations,13546,14694


## Review of Stock Performance 

### Price Actions

In [614]:
# retrieve percentage changes
dfPriceActions, err = ek.get_data(RIC, 'TR.PricePctChg1M/100;TR.PricePctChg3M/100;TR.PricePctChg6M/100;TR.PricePctChg1Y/100')
dfPriceActions

#create lists out of results 
listNames = dfPriceActions.columns.tolist()
listPA = dfPriceActions.iloc[0, :].tolist()

# remove the first element, it's header
listNames.pop(0)
listPA.pop(0)
#display(listNames, listPA)

# Pandas DataFrame from dictionary
dataDictPA = {'Names': listNames, 'Periods': listPA } 
dfPA = pd.DataFrame(dataDictPA, index = listNames) 


import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pyof
pyof.init_notebook_mode(connected=True)

fig = px.bar(dfPA, x='Names', y='Periods')
fig.show()



### Profitability 

In [616]:
dfProfit, err = ek.get_data(RIC, 'TR.GrossMargin;TR.ResearchAndDevelopment;TR.TotalRevenue;TR.SGAExpenseTotal;TR.EBITDAMarginPercent;TR.OperProfitMarginPct;TR.PretaxMarginPercent;TR.NetProfitMargin;TR.CashFlowMarginPercent/100;TR.IncomeTaxRatePct;TR.ROATotalAssetsPercent;TR.ReturnonAvgTotEqtyPctNetIncomeBeforeExtraItems',{'Period':'FY0'})
#dfProfit   
dfProfit2, err = ek.get_data(RIC, 
    'TR.GrossProfitMarginPct5YrAvg();TR.EBITDAMarginPct5YrAvg;TR.OperatingProfitMarginPct5YrAvg;TR.IncomeBefTaxMarginpct5YrAvg();TR.IncomeAftTaxMarginpct5YrAvg();TR.CashFlowMarginPct5YrAvg()/100;TR.IncomeTaxRatePct5YrAvg()/100;TR.ReturnonAvgTotAssetsPctIncomeAfterTax5YrAvg')
dfProfit3, err = ek.get_data(RIC, 'TR.ResearchAndDevelopment(),TR.TotalRevenue(),TR.SGAExpenseTotal,TR.ReturnonAvgTotEqtyPctNetIncomeBeforeExtraItems',{'Period':'FY0','Sdate':0,'Edate':-4,'Frq':'FY'})
dfProfit3['RAD over TR'] = dfProfit3['Research And Development']/dfProfit3['Total Revenue']
dfProfit3['SGA over TR'] = dfProfit3['Selling/General/Administrative Expense, Total']/dfProfit3['Total Revenue']
#dfProfit2

#Create a dictionary out of results, and structure a single multi-row Data Frame
# initialise results per column 
dataDictProfit = {'FY0':
        [dfProfit.at[0,'Gross Margin, Percent'],  dfProfit.at[0,'Research And Development']/dfProfit.at[0,'Total Revenue'],
        dfProfit.at[0,'Selling/General/Administrative Expense, Total']/dfProfit.at[0,'Total Revenue'],
        dfProfit.at[0,'EBITDA Margin, Percent'],dfProfit.at[0,'Operating Profit Margin, Percent'],
        dfProfit.at[0,'Pretax Margin, Percent'],dfProfit.at[0,'Net Profit Margin, (%)'],
        dfProfit.at[0,'TR.CASHFLOWMARGINPERCENT/100'],dfProfit.at[0,'Income Tax Rate, %'],
        dfProfit.at[0,'ROA Total Assets, Percent'],dfProfit.at[0,'ROE Total Equity %']
    ]
        ,'5 Year Avg':[dfProfit2.at[0,'Gross Profit Margin, %, 5 Yr Avg'],dfProfit3['RAD over TR'].mean(),
                       dfProfit3['SGA over TR'].mean(), dfProfit2.at[0,'EBITDA Margin, %, 5 Yr Avg'],
                       dfProfit2.at[0,'Oper Profit Margin, %, 5 Yr Avg'],dfProfit2.at[0,'Income Bef Tax Margin, %, 5 Yr Avg'],
                       dfProfit2.at[0,'Income Aft Tax Margin, %, 5 Yr Avg'],dfProfit2.at[0,'TR.CASHFLOWMARGINPCT5YRAVG()/100'],
                       dfProfit2.at[0,'TR.INCOMETAXRATEPCT5YRAVG()/100'],dfProfit2.at[0,'ROA Tot Assets, %, 5 Yr Avg'],
                       dfProfit3['ROE Total Equity %'].mean()
]
                       
}
#dataDictProfit

# structure as required multi-row
dfMultiProfit = pd.DataFrame(dataDictProfit, index =[
    'Gross Margin','R/D % Sales','SGA % Sales','EBITDA Margin','Operating Margin','Pretax Margin','Net Margin',
    'CF/Sales','Tax Rate','ROA','ROE'
    ]) 
dfMultiProfit   

,FY0,5 Year Avg
Gross Margin,47.296720,47.685373
R/D % Sales,0.077631,0.070337
SGA % Sales,0.252777,0.241042
EBITDA Margin,24.610160,23.801055
Operating Margin,13.177440,15.390266
Pretax Margin,13.177440,15.390517
Net Margin,12.419150,14.292758
CF/Sales,0.202730,0.201579
Tax Rate,5.754476,0.071327
ROA,6.953638,9.292783


### FY0 Sales By Region 

In [562]:
dfSalesByRegion, err = ek.get_data(RIC, 'TR.BGS.GeoTotalRevenue.segmentName;TR.BGS.GeoTotalRevenue()',{'Scale':'6','curn':df.at[0,'Currency']})
dfSalesByRegion

,Instrument,Segment Name,Geographic Total Revenues (Calculated)
0,IBM,Americas,36274
1,IBM,Europe/Middle East/Africa,24443
2,IBM,Asia/Pacific,16430
3,IBM,Segment Total,77147
4,IBM,Consolidated Total,77147


## Price Chart

### Pricing Data  - 1 -  Closing Prices

In [563]:

dfPricingIntermediate1, err = ek.get_data(RIC, 'TR.PriceClose.calcdate;TR.PriceClose',{'sdate':'-5AY','edate':'0d','frq':'D'})
#dfPricingIntermediate1


### Pricing Data - 2 - Dates 

In [564]:
# Intermediate dates to use in the next step
import datetime
from datetime import datetime
from datetime import timedelta

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d)
        d += timedelta(days=1)
    return days

RIC2 = 'MSFT.O'

dfDatesMSFTIntermediate1, err = ek.get_data(RIC2, 'TR.PriceClose.date',{'sdate':'0d'})
dfDatesMSFTIntermediate2, err = ek.get_data(RIC2, 'TR.PriceClose.date',{'sdate':'-24AM'})


# number of work days between the two dates + 250
daysBetween = len(workdays((datetime.strptime(dfDatesMSFTIntermediate2.at[0,'Date'],'%Y-%m-%dT%H:%M:%SZ')),
               (datetime.strptime(dfDatesMSFTIntermediate1.at[0,'Date'],'%Y-%m-%dT%H:%M:%SZ')))) + 250
daysBetween
dfDatesMSFTIntermediate4, err = ek.get_data(RIC2, 'TR.PriceClose.date',{'sdate':'-'+str(daysBetween)+'D'})

display(dfDatesMSFTIntermediate1.at[0,'Date'],
dfDatesMSFTIntermediate2.at[0,'Date'],
dfPricingIntermediate1.at[0,'Calc Date']+'T00:00:00Z',
dfDatesMSFTIntermediate4.at[0,'Date'])

if dfPricingIntermediate1.at[0,'Calc Date']+'T00:00:00Z' > dfDatesMSFTIntermediate2.at[0,'Date']:
    laterDate1 = dfPricingIntermediate1.at[0,'Calc Date']+'T00:00:00Z'
else:
    laterDate1 = dfDatesMSFTIntermediate2.at[0,'Date']
    

if dfPricingIntermediate1.at[0,'Calc Date']+'T00:00:00Z' > dfDatesMSFTIntermediate4.at[0,'Date']:
    laterDate2 = dfPricingIntermediate1.at[0,'Calc Date']+'T00:00:00Z'
else:
    laterDate2 = dfDatesMSFTIntermediate4.at[0,'Date']
    
display(laterDate1,laterDate2)

'2020-03-20T00:00:00Z'

'2018-03-23T00:00:00Z'

'2015-03-23T00:00:00Z'

'2017-02-27T00:00:00Z'

'2018-03-23T00:00:00Z'

'2017-02-27T00:00:00Z'

### Pricing Data - 3 - Volumes, Prices, 50 days, 200 days 

In [565]:
dfDtVolPrice, err = ek.get_data(df_2.at[0,'Primary Quote RIC'], 'TR.PriceClose.CalcDate,TR.PriceClose',{'Sdate':'0D','Edate':laterDate2[:10],'Frq':'D'})

dfDtVolPrice2, err = ek.get_data(df_2.at[0,'Primary Quote RIC'], 'TR.Volume',{'Sdate':'0D','Edate':laterDate2[:10],'Frq':'D','Scale':3})
dfDtVolPrice['Volume'] = dfDtVolPrice2['Volume']

#calculate moving averages
convolve50 = np.convolve(dfDtVolPrice['Price Close'], np.ones((50,))/50, mode='valid')
convolve200 = np.convolve(dfDtVolPrice['Price Close'], np.ones((200,))/200, mode='valid')
display(dfDtVolPrice,convolve50,convolve200)

,Instrument,Calc Date,Price Close,Volume
0,IBM.N,2020-03-20,95.39,3778.055
1,IBM.N,2020-03-19,100.34,2291.137
2,IBM.N,2020-03-18,103.55,2669.332
3,IBM.N,2020-03-17,106.65,2636.786
4,IBM.N,2020-03-16,99.08,3703.656
5,IBM.N,2020-03-13,107.95,3526.853
6,IBM.N,2020-03-12,102.81,4084.483
7,IBM.N,2020-03-11,117.97,2401.283
8,IBM.N,2020-03-10,124.77,3306.631
9,IBM.N,2020-03-09,117.81,3297.583


array([135.5382, 136.3366, 137.0136, 137.6246, 138.1784, 138.9052,
       139.427 , 140.027 , 140.373 , 140.5758, 140.9192, 141.0756,
       141.1964, 141.2032, 141.3134, 141.3118, 141.3914, 141.4134,
       141.3248, 141.1658, 140.9154, 140.597 , 140.237 , 139.858 ,
       139.4758, 139.1042, 138.6762, 138.259 , 137.8648, 137.478 ,
       137.1292, 136.6808, 136.231 , 135.9128, 135.6778, 135.4894,
       135.442 , 135.3682, 135.2668, 135.2062, 135.1044, 134.9992,
       134.8752, 134.8674, 134.859 , 134.8528, 134.831 , 134.7892,
       134.7622, 134.7048, 134.6894, 134.692 , 134.6896, 134.6952,
       134.6876, 134.6308, 134.6318, 134.6608, 134.7976, 134.9594,
       135.1006, 135.2448, 135.3556, 135.4578, 135.5372, 135.6784,
       135.8556, 136.0118, 136.1392, 136.3372, 136.5674, 136.7538,
       136.9604, 137.1854, 137.3792, 137.5782, 137.7576, 137.928 ,
       138.097 , 138.2392, 138.3694, 138.556 , 138.7516, 138.9596,
       139.1702, 139.336 , 139.4594, 139.5988, 139.6356, 139.6

array([137.51255, 137.69305, 137.8548 , 137.9784 , 138.0801 , 138.23255,
       138.34125, 138.4795 , 138.55105, 138.58915, 138.68185, 138.72545,
       138.7533 , 138.7538 , 138.7887 , 138.7892 , 138.805  , 138.79655,
       138.7744 , 138.74255, 138.7004 , 138.6394 , 138.5852 , 138.53215,
       138.4746 , 138.4239 , 138.3537 , 138.2724 , 138.2022 , 138.1232 ,
       138.0559 , 137.9743 , 137.8871 , 137.8432 , 137.8074 , 137.81445,
       137.8501 , 137.8834 , 137.90455, 137.92655, 137.9343 , 137.9369 ,
       137.93385, 137.9519 , 137.9785 , 138.0036 , 138.037  , 138.0634 ,
       138.08   , 138.09275, 138.11015, 138.1295 , 138.1558 , 138.1925 ,
       138.2188 , 138.24415, 138.275  , 138.3081 , 138.3257 , 138.34395,
       138.36045, 138.37125, 138.36875, 138.37275, 138.3856 , 138.4039 ,
       138.4254 , 138.45035, 138.4644 , 138.49145, 138.5205 , 138.5482 ,
       138.57835, 138.608  , 138.63805, 138.67095, 138.69655, 138.7067 ,
       138.72545, 138.73025, 138.72035, 138.7172 , 

### Price Chart 

In [566]:
from plotly.subplots import make_subplots

#fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
#       x=dfDtVolPrice['Calc Date'],  #we can optionally chart the complete set
        x = dfDtVolPrice.loc[:525,['Calc Date']]['Calc Date'],    # we are cutting off
        y=dfDtVolPrice['Price Close'],
        name = 'Price Close'), 
    secondary_y=False,
)
fig.add_trace(go.Scatter(
#    x=dfDtVolPrice['Calc Date'],
    x = dfDtVolPrice.loc[:525,['Calc Date']]['Calc Date'],
    y=dfDtVolPrice['Volume'],
    name = 'Volume'), 
    secondary_y=True,
)
fig.add_trace(go.Scatter(
#    x=dfDtVolPrice['Calc Date'],
    x = dfDtVolPrice.loc[:525,['Calc Date']]['Calc Date'],
    y=convolve50,
    name='50 Day MA',),
    secondary_y=False,
)
fig.add_trace(go.Scatter(
    x = dfDtVolPrice.loc[:525,['Calc Date']]['Calc Date'],
#    x=dfDtVolPrice['Calc Date'],
    y=convolve200,
    name='200 Day MA',),
    secondary_y=False,
)

fig.update_yaxes(title_text="Price", secondary_y=False)
fig.update_yaxes(title_text="Volume", secondary_y=True)

fig.show()

In [625]:
fields = [{'TR.PriceClose;TR.PriceOpen': {'sort_dir':'asc','sort_priority':1}},
          {'TR.PriceOpen': {'sort_dir':'asc','sort_priority':0}}]
data, err = ek.get_data(["IBM","MSFT.O"],fields)


data

,Instrument,Price Close,Price Open,Price Open
0,IBM,112.93,108.09,108.09
1,MSFT.O,149.70,151.75,151.75


In [628]:
dfGen, err = ek.get_data(RIC, 
        [{'TR.PriceClose;TR.Price52WeekHigh;TR.PricePctChg52WkHigh;TR.Price52WeekLow;TR.PricePctChg52WkLow;TR.PriceTargetMedian': {'Curn':df.at[0,'Currency']}},
            {'TR.BetaFiveYear;TR.DividendYield;TR.SharesOutstanding;TR.FreeFloat;TR.FreeFloatPct();TR.CompanyMarketCap;TR.EV': {'Scale':6,'Curn':df.at[0,'Currency']}}])
dfGen

,Instrument,Price Close,Price 52 Week High,52-week High Price PCT Change,Price 52 Week Low,52-week Low Price PCT Change,Price Target - Median,Beta 5 Year,Dividend yield,Outstanding Shares,Free Float,Free Float (Percent),Company Market Cap,Enterprise Value (Daily Time Series)
0,IBM,112.93,158.75,-28.862992,90.56,24.701855,139,1.214492,5.998334,888408023,887346656,99.8805,1.003279e+11,1.545029e+11
